# 1. Carregamento das bibliotecas 📦

In [ ]:
!pip install PyGithub
!pip install Unidecode
!python -m spacy download pt_core_news_sm
import pandas as pd
import re
import unidecode
import spacy
import getpass
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.over_sampling import SMOTE
from github import Github
import base64

# 2. Código Machine Learning ⚙️

In [ ]:
# Carragamento dos dados
github_csv_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/silver/order_reviews_tratados.csv"
df = pd.read_csv(github_csv_url)

# Pré-processamento do texto
nlp = spacy.load("pt_core_news_sm")

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = unidecode.unidecode(text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    doc = nlp(text)
    stop_words = {
        'o', 'a', 'os', 'as', 'de', 'do', 'da', 'em', 'por', 'para', 'com',
        'é', 'está', 'etc', 'que', 'e', 'no', 'na', 'um', 'uma', 'foi',
        'tá', 'tava', 'pra', 'mas', 'como', 'se', 'tem', 'já', 'bem',
        'muito', 'vai', 'ser'
    }
    tokens = [token.lemma_ for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 2]
    return ' '.join(tokens)

# 3. Classificação dos sentimentos
def classify_sentiment(score):
    if score >= 4:
        return 'positivo'
    elif score == 3:
        return 'neutro'
    else:
        return 'negativo'

df['review_comment_title'] = df['review_comment_title'].fillna('')
df['review_comment_message'] = df['review_comment_message'].fillna('')
df['combined_text'] = df['review_comment_title'] + ' ' + df['review_comment_message']
df['sentiment'] = df['review_score'].apply(classify_sentiment)

# Separação entre treino e teste
X = df['combined_text']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Pipeline de processamento e classificação
pipeline = make_imb_pipeline(
    TfidfVectorizer(
        preprocessor=preprocess_text,
        ngram_range=(1, 3),
        max_features=8000,
        min_df=5
    ),
    SMOTE(sampling_strategy='not majority', random_state=42),
    LinearSVC(class_weight='balanced', C=1.0, max_iter=10000, random_state=42)
)

# Treinamento do modelo
pipeline.fit(X_train, y_train)

# 7. Avaliação do modelo
y_pred = pipeline.predict(X_test)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Previsões finais e exportação
df['predicted_sentiment'] = pipeline.predict(df['combined_text'])
output_file = "order_reviews_com_sentimento.csv"
df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n Arquivo exportado como {output_file}")

# Função de previsão manual
def predict_sentiment(text):
    return pipeline.predict([text])[0]

print("\nTestes manuais:")
print("-> Produto excelente, recomendo!:", predict_sentiment("Produto excelente, recomendo!"))
print("-> A entrega atrasou mas o produto é bom:", predict_sentiment("A entrega atrasou mas o produto é bom"))
print("-> Muito ruim, veio quebrado:", predict_sentiment("Muito ruim, veio quebrado"))

# 10. Upload para GitHub
print("\n🔒 Autenticação no GitHub")
token = getpass.getpass("Digite seu token de acesso do GitHub (ele não será exibido): ")
repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
remote_path = "projeto/development/gold/order_reviews_com_sentimento.csv"
commit_message = "Adiciona arquivo com sentimentos previstos"

# Conectar ao GitHub e enviar o arquivo
try:
    g = Github(token)
    repo = g.get_repo(repo_name)

    with open(output_file, "rb") as file:
        content = file.read()
        content_b64 = base64.b64encode(content).decode("utf-8")

    try:
        existing_file = repo.get_contents(remote_path)
        repo.update_file(remote_path, commit_message, content, existing_file.sha)
        print(" Arquivo atualizado no GitHub com sucesso!")
    except:
        repo.create_file(remote_path, commit_message, content, branch="main")
        print(" Arquivo criado no GitHub com sucesso!")

except Exception as e:
    print(f" Erro ao enviar para o GitHub: {e}")